In [16]:
#Python
#import module for predict
import os, glob
import numpy as np
import configparser
import cv2
from matplotlib import pyplot as plt
#Keras
from keras.models import model_from_json
from keras.models import Model
import keras.backend
#scikit learn
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
import sys
sys.path.insert(0, './lib/')
# help_functions.py
from help_functions import *
# extract_patches.py
from extract_patches import *
# pre_processing.py
from pre_processing import *

from keras.models import Model


In [17]:

import cv2
import numpy as np
import matplotlib.pyplot
def show_on_jupyter(img,color= None,title=None):
    import matplotlib.pyplot as plt
    import cv2
    """Show img on jupyter notebook. No return Value
    
    You should check the img's color space.
    I just consider about RGB color space & 1 ch color space(like green ch, gray space, ...)
    
    using matplotlib
    
    Parameters
    ----------
    img : 2-D Array
        numpy 2-D array
        opencv / sklearn / plt are avaliable.
        float / uint8 data type.
        
    color : string
        'gray' or 'None'
        'gray' means that img has a 1 ch.
        'None' means that img has a RGB ch.
        (default: None)
        
    title : string
        decide img's title
        (default : None)
        
    Returns
    -------
        No return value.
    
    Example
    -------
    >>> img = cv2.imread(img_path)
    >>> show_on_jupyter(img)
    
    img has a 1 ch
    >>> img = cv2.imread(img_path)
    >>> show_on_jupyter(img,'gray')
    """
    if color == 'gray':
        plt.axis("off")
        plt.title(title)
        plt.axis('off')
        plt.figure(figsize=(10, 10))
        plt.imshow(img,cmap=color)
        plt.show()
    elif color == None:
        plt.axis("off")
        plt.title(title)
        plt.axis('off')
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()
    else:
        print("Gray or None")


def load_data(data_path):
    
    imgs = np.empty((NUM_IMG,1,HEIGHT_IMG,WIDTH_IMG))
    cnt = 0
    for path, subdirs, files in os.walk(data_path): #list all files, directories in the path
        for i in range(len(files)):
            tempImg = Image.open(data_path + files[i])
            #print(files[i])
            #tempImg = np.asarray(tempImg)
            #print(tempImg.shape)
            w, h = tempImg.size
            print(w,h)
            print('w,h : ', w,'x',h)
            if (w == 700 and h ==380):
                print("Hi")
                tempImg = np.asarray(tempImg)
                tempImg = tempImg[np.newaxis,:,:]
                imgs[cnt] = tempImg
                cnt += 1
            
    return imgs

def group_plot(data,row,col):
    fig=plt.figure(figsize=(13, 13))
    columns = col
    rows = row
    for num in range(len(data)):
        img = data[num]
        fig.add_subplot(rows, columns, num+1)
        plt.imshow(img,cmap='gray')
    plt.show()

def get_conjunctivaData_testing(Conjunctiva_test_img_ori, num_test_img,patch_h, patch_w):
    
    # get img data
    test_img_ori = Conjunctiva_test_img_ori
    
    test_imgs = my_gray_preprocessing(test_img_ori)
    
    # extend both images and masks so they can be divided exactly by the patches dimensions
    # make tensor data.
    
    test_imgs = test_imgs[0:num_test_img,:,:,:]
    print('type : ',type(test_imgs))
    print('[get data testing func] prev test img shape : {} '.format(test_imgs.shape))
    test_imgs = paint_border(test_imgs,patch_h, patch_w)
    print('[get data testing func] after test img shape : {} '.format(test_imgs.shape))

    #data_consistency_check(test_imgs, test_grds)
    #assert(np.max(test_grds)==1  and np.min(test_grds)==0)
    
    print ("[get_data_testing_func] test images range (min-max): " +str(np.min(test_imgs)) +' - '+str(np.max(test_imgs)))
    print ("[get_data_testing_fucn] test masks are within 0-1\n")
    
def width_padding(img, pad_size):
    h,w = np.shape(img)
    #print('pad size', pad_size)
    
    if pad_size % 2 ==1: #odd number
        pad01 = np.zeros((h, pad_size//2))
        pad02 = np.zeros((h, pad_size//2 +1))
    
        result = np.hstack([pad01, img])
        result = np.hstack([result, pad02])
        
    else:
        pad01 = np.zeros((h, pad_size//2))
    
        result = np.hstack([pad01, img])
        result = np.hstack([result, pad01])
        
    print(np.shape(img), ' ',np.shape(result))
    return result
    
def height_padding(img,pad_size):
    h,w = np.shape(img)
    #print('pad size : ',pad_size)
    
    if pad_size % 2 ==1: #odd number
        pad01 = np.zeros((pad_size//2,w ))
        pad02 = np.zeros((pad_size//2 +1 , w))
        
        result = np.vstack([pad01, img])
        result = np.vstack([result,pad02])
        
    else:
        pad01 = np.zeros((pad_size//2, w))
    
        result = np.vstack([pad01, img])
        result = np.vstack([result, pad01])
    print(np.shape(img), ' ',np.shape(result))
    return result

    patches_imgs_test = extract_ordered(test_imgs,patch_h,patch_w)
    #data_consistency_check(test_imgs, test_grds)

    print ("\n[get_data_testing_fucn] test PATCHES images/grds shape:")
    print (patches_imgs_test.shape)
    print ("[get_data_testing_fucn] test PATCHES images range (min-max): {} - {}".format(str(np.min(patches_imgs_test)), str(np.max(patches_imgs_test))))

    return patches_imgs_test

def get_conjunctivaData_testing_overlap(Conjunctiva_test_img_ori, num_test_img,
                                        patch_h,patch_w,stride_h, stride_w):
    
    # get img data
    test_img_ori = Conjunctiva_test_img_ori
    print(np.shape(test_img_ori))
    test_imgs = my_gray_preprocessing(test_img_ori)
    
    # extend both images and masks so they can be divided exactly by the patches dimensions
    # make tensor data.
    
    test_imgs = test_imgs[0:num_test_img,:,:,:]
    
    #print('type : ',type(test_imgs))
    #print('[get data testing overlap] prev test img shape : {}  '.format(test_imgs.shape))
    test_imgs = paint_border_overlap(test_imgs,patch_h, patch_w,stride_h, stride_w)
    #test_grds = paint_border_overlap(test_grds, patch_h, patch_w,stride_h, stride_w)
    #print('[get data testing overlap] after test img shape : {}'.format(test_imgs.shape))
    
    #print ("[get_data_testing_overlap func] test images range (min-max): " +str(np.min(test_imgs)) +' - '+str(np.max(test_imgs)))
    #print ("[get_data_testing_overlap fucn] test masks are within 0-1\n")
    
    patches_imgs_test = extract_ordered_overlap(test_imgs,patch_h,patch_w,stride_h,stride_w)
    
    #print ("\n[get_data_testing_overlap func] test PATCHES images shape:")
    print (patches_imgs_test.shape)
    #print ("[get_data_testing_overlap func] test PATCHES images range (min-max): " +str(np.min(patches_imgs_test)) +' - '+str(np.max(patches_imgs_test)))

    return patches_imgs_test, test_imgs.shape[2], test_imgs.shape[3]

## 여기부터 시작

In [18]:
#========= CONFIG FILE TO READ FROM =======
config = configparser.RawConfigParser()
config.read('conj_configuration.txt')

#run the training on invariant or local
path_data = config.get('data paths', 'conj_path_local')

patch_height = int(config.get('data attributes', 'patch_height'))
patch_width = int(config.get('data attributes', 'patch_width'))

stride_height = int(config.get('testing settings', 'stride_height'))
stride_width = int(config.get('testing settings', 'stride_width'))

name_experiment = config.get('experiment name', 'name')
path_experiment = './' +name_experiment +'/'
average_mode = config.getboolean('testing settings', 'average_mode')

target_experiment = config.get('testing settings','experiment_target')
target_path = './' + target_experiment + '/'

if os.path.isdir('./'+name_experiment) == False:
    os.mkdir('./'+name_experiment)
else:
    print('already exist the folder in this path : {}'.format('./'+name_experiment))

print(path_data)
print(target_experiment, '\n', target_path)
print(name_experiment)
print(path_experiment)

./conj_train_avg/
DRIVE_test_norm_150000 
 ./DRIVE_test_norm_150000/
conjunctival_avg_result
./conjunctival_avg_result/


In [19]:
filst = sorted(glob.glob('./'+name_experiment))

In [20]:
HEIGHT_IMG = 584
WIDTH_IMG = 565
NUM_IMG = 0
CONJ_IMG_LIST = []

print(path_data)

for path, subdirs, files in sorted(os.walk(path_data)): #list all files, directories in the path
        #files.sort()
        #print('sub  : ',subdirs)
        for i in range(len(sorted(files))):
            print(i)
            print(path_data+files[i])
            tempImg = cv2.imread(path_data + files[i],0)
            print(np.shape(tempImg))
            CONJ_IMG_LIST.append(tempImg)
        
            NUM_IMG = NUM_IMG +1
            
CONJ_IMG_LIST = np.asarray(CONJ_IMG_LIST)
print(np.shape(CONJ_IMG_LIST))
print(type(CONJ_IMG_LIST))

./conj_train_avg/
0
./conj_train_avg/train_data_24.png
(584, 565)
1
./conj_train_avg/train_data_5.png
(584, 565)
2
./conj_train_avg/train_data_34.png
(584, 565)
3
./conj_train_avg/train_data_1.png
(584, 565)
4
./conj_train_avg/train_data_17.png
(584, 565)
5
./conj_train_avg/train_data_11.png
(584, 565)
6
./conj_train_avg/train_data_8.png
(584, 565)
7
./conj_train_avg/train_data_14.png
(584, 565)
8
./conj_train_avg/train_data_4.png
(584, 565)
9
./conj_train_avg/train_data_19.png
(584, 565)
10
./conj_train_avg/train_data_25.png
(584, 565)
11
./conj_train_avg/train_data_13.png
(584, 565)
12
./conj_train_avg/train_data_22.png
(584, 565)
13
./conj_train_avg/train_data_7.png
(584, 565)
14
./conj_train_avg/train_data_18.png
(584, 565)
15
./conj_train_avg/train_data_15.png
(584, 565)
16
./conj_train_avg/train_data_32.png
(584, 565)
17
./conj_train_avg/train_data_28.png
(584, 565)
18
./conj_train_avg/train_data_31.png
(584, 565)
19
./conj_train_avg/train_data_9.png
(584, 565)
20
./conj_train_av

In [21]:
temp = np.expand_dims(CONJ_IMG_LIST,axis =1)
print(np.shape(temp))
temp = np.asarray(temp)
print(type(temp))
print(np.shape(temp))

(34, 1, 584, 565)
<class 'numpy.ndarray'>
(34, 1, 584, 565)


In [22]:
#conj_img = load_data(path_data) 
conj_img = np.expand_dims(CONJ_IMG_LIST,axis =1)

if average_mode == True:
    patches_conj_imgs_test, new_height, new_width = get_conjunctivaData_testing_overlap(conj_img,
                                                         NUM_IMG,
                                                         patch_height,
                                                         patch_width,
                                                         stride_height,
                                                        stride_width)
    conj_totimg = visualize(group_images(patches_conj_imgs_test[100:200,:,:,:],2),path_experiment+'conj_overlap_test_patch_img')
else:
    patches_conj_imgs_test = get_conjunctivaData_testing(conj_img,
                                                         NUM_IMG,
                                                         patch_height,
                                                         patch_width)
    
    conj_totimg = visualize(group_images(patches_conj_imgs_test[100:200,:,:,:],2),path_experiment+'conj_test_patch_img')


(34, 1, 584, 565)

the side H is not compatible with the selected stride of 5
img_h 584, patch_h 48, stride_h 5
(img_h - patch_h) MOD stride_h: 1
So the H dim will be padded with additional 4 pixels

the side H is not compatible with the selected stride of 5
img_h 565, patch_h 48, stride_h 5
(img_h - patch_h) MOD stride_h: 2
So the H dim will be padded with additional 3 pixels
new full images shape: 
(34, 1, 588, 568)
extract , :  540 5 520 5
[extrct_order_overlap func] Number of patches on h : 109
[extrct_order_overlap func] Number of patches on w : 105
[extrct_order_overlap func] number of patches per image: 11445 totally for this dataset: 389130
(389130, 1, 48, 48)
[group images func] prev data shape  : (100, 1, 48, 48)
[group images func] after data shape :  (100, 48, 48, 1)
[group images func] first total image :  (48, 96, 1)
[group images func] final total image :  (2448, 96, 1)
data shape :  (2448, 96, 1)
<PIL.Image.Image image mode=L size=96x2448 at 0x7F24EC2AE898>
file name : 

In [23]:
#================ Run the prediction of the patches ==================================
best_last = config.get('testing settings', 'best_last')
#Load the saved model
model = model_from_json(open(target_path +target_experiment+'_architecture.json').read())
model.load_weights(target_path+best_last+'_weights.h5')




In [24]:
print(target_path +target_experiment+'_architecture.json')
print(target_path,'\n', best_last, '    _')
print(model)

./DRIVE_test_norm_150000/DRIVE_test_norm_150000_architecture.json
./DRIVE_test_norm_150000/ 
 best     _


In [25]:
print(np.shape(patches_conj_imgs_test))

temp_conj_patchs = patches_conj_imgs_test[: 50000,:,:,:]
print(np.shape(temp_conj_patchs))

(389130, 1, 48, 48)
(50000, 1, 48, 48)


In [26]:
conj_predictions = model.predict(patches_conj_imgs_test, batch_size=32, verbose=1)

#conj_predictions = model.predict(temp_conj_patchs, batch_size=128, verbose=2)


389130/389130 [==============================] - 261s 670us/step


In [27]:
pred_conj_patches = pred_to_imgs(conj_predictions, patch_height, patch_width, "original")
pred_conj_patches_thresh = pred_to_imgs(conj_predictions, patch_height, patch_width, "threshold")


temp5_totimg = visualize(group_images(pred_conj_patches[150:200,:,:,:],2),path_experiment+'/predict_conj_patch_ori')
temp6_totimg = visualize(group_images(pred_conj_patches_thresh[150:200,:,:,:],2),path_experiment+'/predict_conj_patch_thresh')


[group images func] prev data shape  : (50, 1, 48, 48)
[group images func] after data shape :  (50, 48, 48, 1)
[group images func] first total image :  (48, 96, 1)
[group images func] final total image :  (1248, 96, 1)
data shape :  (1248, 96, 1)
<PIL.Image.Image image mode=L size=96x1248 at 0x7F24DC911A20>
file name :  ./conjunctival_avg_result//predict_conj_patch_ori
[group images func] prev data shape  : (50, 1, 48, 48)
[group images func] after data shape :  (50, 48, 48, 1)
[group images func] first total image :  (48, 96, 1)
[group images func] final total image :  (1248, 96, 1)
data shape :  (1248, 96, 1)
<PIL.Image.Image image mode=L size=96x1248 at 0x7F24DC97AFD0>
file name :  ./conjunctival_avg_result//predict_conj_patch_thresh


In [28]:
if average_mode == True:
    conj_pred_imgs = recompose_overlap_img(pred_conj_patches, new_height, new_width, stride_height, stride_width)# predictions
    conj_pred_imgs_thr = recompose_overlap_img(pred_conj_patches_thresh, new_height, new_width, stride_height, stride_width)# predictions
    conj_ori_imgs = recompose_overlap_img(patches_conj_imgs_test,new_height, new_width, stride_height, stride_width)
   
    tempConj01_totimg = visualize(group_images(conj_pred_imgs[:,:,:,:],2),path_experiment+'/conj_predict_imgs')
    tempConj02_totimg = visualize(group_images(conj_pred_imgs_thr[:,:,:,:],2),path_experiment+'/conj_predict_thresh_imgs')
    tempConj03_totimg = visualize(group_images(conj_ori_imgs[:,:,:,:],2), path_experiment+'/conj_ori_imgs')


else:
    conj_pred_imgs = recompose_img(pred_conj_patches,num_patch_per_img_h,num_patch_per_img_w)
    conj_pred_imgs_thresh = recompose_img(pred_conj_patches_thresh,num_patch_per_img_h,num_patch_per_img_w)
    conj_ori_imgs = recompose_img(patches_conj_imgs_test,num_patch_per_img_h,num_patch_per_img_w)

    tempConj01_totimg = visualize(group_images(conj_pred_imgs[:,:,:,:],2),path_experiment+'/conj_predict_imgs')
    tempConj02_totimg = visualize(group_images(conj_pred_imgs_thresh[:,:,:,:],2),path_experiment+'/conj_predict_thresh_imgs')
    tempConj03_totimg = visualize(group_images(conj_ori_imgs[:,:,:,:],2), path_experiment+'/conj_ori_imgs')


N_patches_h: 109
N_patches_w: 105
N_patches_img: 11445
According to the dimension inserted, there are 34 full images (of 588x568 each)
(34, 1, 588, 568)
N_patches_h: 109
N_patches_w: 105
N_patches_img: 11445
According to the dimension inserted, there are 34 full images (of 588x568 each)
(34, 1, 588, 568)
N_patches_h: 109
N_patches_w: 105
N_patches_img: 11445
According to the dimension inserted, there are 34 full images (of 588x568 each)
(34, 1, 588, 568)
[group images func] prev data shape  : (34, 1, 588, 568)
[group images func] after data shape :  (34, 588, 568, 1)
[group images func] first total image :  (588, 1136, 1)
[group images func] final total image :  (10584, 1136, 1)
data shape :  (10584, 1136, 1)
<PIL.Image.Image image mode=L size=1136x10584 at 0x7F24DC962320>
file name :  ./conjunctival_avg_result//conj_predict_imgs
[group images func] prev data shape  : (34, 1, 588, 568)
[group images func] after data shape :  (34, 588, 568, 1)
[group images func] first total image :  (5

In [29]:
print(np.shape(conj_pred_imgs_thr[3,0,:,:]))

(588, 568)


In [34]:
# make each prediction images

print(np.shape(conj_pred_imgs_thr))

if os.path.isdir(path_experiment + 'predicted_imgs') == False:
    os.mkdir(path_experiment + 'predicted_imgs')
else:
    print('already exist the folder in this path : {}'.format(path_experiment + 'predicted_imgs'))

temp_predicted_imgs = None

for i in range(len(conj_pred_imgs_thr)):
    temp_predicted_imgs = conj_pred_imgs_thr[i,0,:,:] * 255
    temp_save_imgs = Image.fromarray(temp_predicted_imgs.astype(np.uint8))
    temp_save_imgs.save(path_experiment+'predicted_imgs/'+str(i)+'_predict.png')

(34, 1, 588, 568)
already exist the folder in this path : ./conjunctival_avg_result/predicted_imgs


In [31]:
temp = conj_pred_imgs_thr[0,0,:,:]
temp_stack = np.dstack([temp,temp])
temp_stack = np.dstack([temp_stack,temp])

print(np.shape(temp_stack))

(588, 568, 3)


In [32]:
print(np.average(conj_pred_imgs_thr))

0.0412045237008473


In [33]:
# make each prediction images

print(np.shape(conj_pred_imgs_thr))

if os.path.isdir(path_experiment + 'predicted_imgs') == False:
    os.mkdir(path_experiment + 'predicted_imgs')
else:
    print('already exist the folder in this path : {}'.format(path_experiment + 'predicted_imgs'))

temp_predicted_imgs = None

for i in range(len(conj_pred_imgs_thr)):
    temp_predicted_imgs = conj_pred_imgs_thr[i,0,:,:] * 255
    
    temp = np.copy(np.array(temp_predicted_imgs,dtype='uint8'))

    #print(np.shape(temp))
    
    
    temp_stack = np.dstack([temp,temp])
    temp_stack = np.dstack([temp_stack,temp])
    #print('temp_stack', np.shape(temp_stack))
    
    temp_cvt_stack = cv2.cvtColor(temp_stack, cv2.COLOR_RGB2RGBA)
    temp_cvt_stack[np.all(temp_cvt_stack == [0, 0, 0, 255], axis=2)] = [0, 0, 0, 0]
    
    #print('temp_cvt_stack : ',np.shape(temp_cvt_stack))
    
    temp_save_imgs = Image.fromarray(temp_cvt_stack.astype(np.uint8))
    temp_save_imgs.save(path_experiment+'predicted_imgs/'+str(i)+'_predict.png')
    


(34, 1, 588, 568)
already exist the folder in this path : ./conjunctival_avg_result/predicted_imgs
